## NOTE
The Purpose of my repos is to learn, in this repo I try to learn how to use TensorFlow Keras, Hyperopt, MLflow to develop a deep learning model

It includes the following steps: 
- STEP 1: DATA PREP
Load and preprocess data 
- STEP 2: Neural Network Model
        Part 1. Create a neural network model with TensorFlow Keras and view training with inline TensorBoard
        Part 2. Perform automated hyperparameter tuning with Hyperopt and MLflow and use autologging to save results
        Part 3. Use the best set of hyperparameters to build a final model
        Part 4. Register the model in MLflow and use the model to make predictions

This repo follow instructions notebooks provided on DataBricks websites

In [2]:
#tensorflow libraries, mlflow
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
import mlflow
import mlflow.keras
import mlflow.tensorflow

## DATA PREP
Using California Housing dataset scikit-learn.

### Load and train-test-split dataset

In [7]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

df_cal_housing = fetch_california_housing()

#split train-test
X_train, X_test, y_train, y_test = train_test_split(df_cal_housing.data, df_cal_housing.target, test_size=0.2)

In [10]:
X_train

array([[   2.1447    ,   26.        ,    3.48595041, ...,    3.50413223,
          33.79      , -118.27      ],
       [   4.1739    ,   36.        ,    5.45962733, ...,    2.86749482,
          34.09      , -117.71      ],
       [   3.1641    ,   17.        ,    5.58348624, ...,    2.6733945 ,
          35.35      , -120.49      ],
       ...,
       [   2.63      ,   52.        ,    4.51311953, ...,    3.29737609,
          34.07      , -117.75      ],
       [   3.3326    ,   52.        ,    3.89162562, ...,    3.60098522,
          34.12      , -118.2       ],
       [   8.0784    ,   52.        ,    6.88219178, ...,    2.67945205,
          33.77      , -117.87      ]])

### Scale features
Feature scaling is important when working with neural networks, we will use StandardScaler

In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test) #scaler is fitted by X_train already, using transform only

## Neural Network Model

#### Part 1. Create model and view TensorBoard in notebook

In [12]:
#define model
def create_model():
    model = Sequential()
    #relu = rectified linear activation function - looks and acts like a linear function, 
    #but is, in fact, a nonlinear function allowing complex relationships in the data to be learned.
    #Dense is layer, is deeply connected with its preceding layer 
    # which means the neurons of the layer are connected to every neuron of its preceding layer. 
    model.add(Dense(20, input_dim=8, activation='relu'))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(1, activation='linear'))
    return model

In [13]:
#compile the model
model = create_model()
#----
model.compile(loss='mse',
             optimizer='Adam',
             metrics=['mse'])

Metal device set to: Apple M1


2023-03-01 23:55:34.519957: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-01 23:55:34.520184: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [ ]:
#